# Recommendation System with GNNs: Examining Vanilla GNN, LightGCN, and GraphSage.
## Envrionment Setup and Dataset

In [8]:
import os
import torch
import torch_geometric
from torch_geometric.datasets import Amazon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import seaborn as sns
import torch_geometric.transforms as T


In [9]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [11]:
dataset = Amazon(root='data/Amazon', name='Computers', 
                transform=T.TargetIndegree())
data = dataset[0]

Processing...
Done!


https://pytorch-geometric.readthedocs.io/en/2.5.3/generated/torch_geometric.datasets.Amazon.html

In [12]:
print(data,end="\n\n")

print("Dataset classes = " + str(dataset.num_classes))
print("Labels in [" + str(int(min(data.y))) + ", " + str(int(max(data.y))) + "]")
print("data.y = ",end="")
print(data.y,end="\n\n")

print("Number of Nodes = " + str(data.num_nodes))
print("Features per Node = " + str(dataset.num_node_features))
print("data.x = ", end="")
print(data.x,end="\n\n")

print("Number of edges = " + str(data.num_edges))
print("Features per edge = " + str(dataset.num_edge_features))
print("data.edge_index = ", end="")
print(data.edge_index)
print("data.edge_attr = ",end="")
print(data.edge_attr)

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752], edge_attr=[491722, 1])

Dataset classes = 10
Labels in [0, 9]
data.y = tensor([4, 4, 8,  ..., 8, 4, 0])

Number of Nodes = 13752
Features per Node = 767
data.x = tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 1., 1., 0.]])

Number of edges = 491722
Features per edge = 1
data.edge_index = tensor([[    0,     0,     0,  ..., 13751, 13751, 13751],
        [  507,  6551,  8210,  ..., 12751, 13019, 13121]])
data.edge_attr = tensor([[0.0471],
        [0.1029],
        [0.8382],
        ...,
        [0.0431],
        [0.0411],
        [0.0087]])


https://github.com/SimonIyamu/Geometric-Deep-Learning/blob/master/GCN_Amazon_Products_Classification.ipynb

## Theory

GNN Frameworks generally use an aggregator function and an update function. From *Graph Neural Networks in Recommender Systems: A Survey* by S.Wu et al, These functions are formally defined as:
___
![Formal Definition of GNN](./img/notebook/formal_def_gnn.png)
___
We Seek to explore additional architectures (GCN, GraphSAGE), which define different aggregation and update functions. Again from S.Wu et al.:
___
![GCN Aggregation and GraphSage Aggregator/Update](./img/notebook/formal_def_gnc_graphsage.png)
___